# Remove Tables if Exists

In [1]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('events.db')
cursor = conn.cursor()

# Drop the tables if they exist
cursor.execute('''
DROP TABLE IF EXISTS Event;
''')

cursor.execute('''
DROP TABLE IF EXISTS People;
''')

cursor.execute('''
DROP TABLE IF EXISTS Event_Participants;
''')

# Commit the changes and close the connection
conn.commit()

# Table Creation

In [2]:

# Create the Event Table with at least 7 columns
cursor.execute('''
CREATE TABLE IF NOT EXISTS Event (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    EventName TEXT NOT NULL,
    EventDate DATE NOT NULL,
    Location TEXT,
    Duration INTEGER,  -- Duration in hours
    Category TEXT,     -- Type of event (e.g., conference, concert)
    Organizer TEXT     -- Event organizer's name or company
);
''')

# Create the People Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS People (
    PersonID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName TEXT NOT NULL,
    LastName TEXT NOT NULL,
    Gender TEXT NOT NULL,
    Age INTEGER,      -- Age of the person
    Email TEXT,       -- Email address
    PhoneNumber TEXT  -- Contact number
);
''')

# Create the Event_Participants Table (linking People and Event)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Event_Participants (
    EventID INTEGER,
    PersonID INTEGER,
    Role TEXT,        -- Role of the person in the event (e.g., speaker, attendee)
    FOREIGN KEY (EventID) REFERENCES Event (EventID),
    FOREIGN KEY (PersonID) REFERENCES People (PersonID),
    PRIMARY KEY (EventID, PersonID)
);
''')

# Commit the changes and close the connection
conn.commit()


# Inserting data into Event Table

In [3]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load the CSV data into a DataFrame
df = pd.read_csv('random_name.csv')



# Generate sample event data based on the CSV data
event_names = ['Tech Conference', 'Music Festival', 'Art Exhibition', 'Business Seminar']
locations = ['Berlin', 'London', 'Paris', 'Munich']
categories = ['Conference', 'Festival', 'Seminar', 'Exhibition']
organizers = ['TechCorp', 'MusicEvents', 'ArtWorld', 'GlobalBusiness']
start_date = datetime(2025, 1, 1)

# Insert Event data based on CSV file entries
for index, row in df.iterrows():
    # Generate unique event data for each entry in the CSV
    event_name = random.choice(event_names)
    event_date = start_date + timedelta(days=random.randint(0, 365))  # Random date within a year
    location = random.choice(locations)
    duration = random.randint(1, 8)  # Random duration in hours
    category = random.choice(categories)
    organizer = random.choice(organizers)
    
    cursor.execute('''
    INSERT INTO Event (EventName, EventDate, Location, Duration, Category, Organizer)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (event_name, event_date.strftime('%Y-%m-%d'), location, duration, category, organizer))

# Commit the changes and close the connection
conn.commit()


# Inserting Data Into People table

In [4]:

# Insert random data for the People table
for index, row in df.iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']
    gender = row['Gender']
    people_count = row['People']  # The number of people associated with the person

    # Random age between 18 and 70
    age = random.randint(18, 70)
    
    # Generate a random email address using the person's name
    email = f"{first_name.lower()}.{last_name.lower()}@example.com"
    
    # Generate a random 10-digit phone number (e.g., 555-123-4567)
    phone_number = f"555-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
    
    # Insert the data into the People table
    cursor.execute('''
    INSERT INTO People (FirstName, LastName, Gender, Age, Email, PhoneNumber)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (first_name, last_name, gender, age, email, phone_number))

# Commit the changes and close the connection
conn.commit()



# Inserting Data Into Event_participants table

In [5]:

# Fetch all EventIDs from the Event table
cursor.execute('SELECT EventID FROM Event')
event_ids = cursor.fetchall()

# Fetch all PersonIDs from the People table
cursor.execute('SELECT PersonID FROM People')
person_ids = cursor.fetchall()

# Set a limit for the number of records to insert (3000 to 4000)
record_limit = random.randint(3000, 4000)

# Ensure we insert no more than the record limit
inserted_records = 0
while inserted_records < record_limit:
    # Randomly select an EventID and PersonID
    event_id = random.choice(event_ids)[0]
    person_id = random.choice(person_ids)[0]
    
    # Randomly assign a role of 'Attendee' or 'Speaker'
    role = random.choice(['Attendee', 'Speaker'])
    
    # Check if the combination of EventID and PersonID already exists
    cursor.execute('''
    SELECT 1 FROM Event_Participants WHERE EventID = ? AND PersonID = ?
    ''', (event_id, person_id))
    
    existing_record = cursor.fetchone()
    
    # If the combination does not exist, insert the new record
    if existing_record is None:
        cursor.execute('''
        INSERT INTO Event_Participants (EventID, PersonID, Role)
        VALUES (?, ?, ?)
        ''', (event_id, person_id, role))
        
        # Increment the count of inserted records
        inserted_records += 1

# Commit the changes and close the connection
conn.commit()
conn.close()


In [7]:
conn.close()